In [11]:
from dash import Dash, html, dcc
from dash.dependencies import Output, Input

from dash_extensions import Lottie       # pip install dash-extensions
import dash_bootstrap_components as dbc  # pip install dash-bootstrap-components
import plotly.express as px              # pip install plotly
import pandas as pd                      # pip install pandas
from datetime import date
import calendar
from wordcloud import WordCloud          # pip install wordcloud

# Lottie by Emil - https://github.com/thedirtyfew/dash-extensions
url_connections = "https://assets9.lottiefiles.com/private_files/lf30_5ttqPi.json"
url_companies = "https://assets9.lottiefiles.com/packages/lf20_EzPrWM.json"
url_msg_in = "https://assets9.lottiefiles.com/packages/lf20_8wREpI.json"
url_msg_out = "https://assets2.lottiefiles.com/packages/lf20_Cc8Bpg.json"
url_reactions = "https://assets2.lottiefiles.com/packages/lf20_nKwET0.json"
options = dict(loop=True, autoplay=True, rendererSettings=dict(preserveAspectRatio='xMidYMid slice'))

# Import App data from csv sheets **************************************
df_cnt = pd.read_csv(r"C:\Users\abhiy\OneDrive\Desktop\project\linked in data\connections.csv", delimiter=",", skipinitialspace=True, on_bad_lines='skip')
df_cnt["Connected On"] = pd.to_datetime(df_cnt["Connected On"], format="%d-%b-%y", errors='coerce')
df_cnt["month"] = df_cnt["Connected On"].dt.month
df_cnt['month'] = df_cnt['month'].apply(lambda x: calendar.month_abbr[x])

df_invite = pd.read_csv(r"C:\Users\abhiy\OneDrive\Desktop\project\linked in data\invitations.csv", delimiter=",", skipinitialspace=True, on_bad_lines='skip')
df_invite["Sent At"] = pd.to_datetime(df_invite["Sent At"], format="%d-%b-%y", errors='coerce')

df_react = pd.read_csv(r"C:\Users\abhiy\OneDrive\Desktop\project\linked in data\profile.csv", delimiter=",", skipinitialspace=True, on_bad_lines='skip')
df_react["Date"] = pd.to_datetime(df_react["Date"], format="%d-%b-%y", errors='coerce')

df_msg = pd.read_csv(r"C:\Users\abhiy\OneDrive\Desktop\project\linked in data\messages.csv", delimiter=",", skipinitialspace=True, on_bad_lines='skip')
df_msg["DATE"] = pd.to_datetime(df_msg["DATE"], format="%d-%b-%y", errors='coerce')

# Bootstrap themes by Ann: https://hellodash.pythonanywhere.com/theme_explorer
app = Dash(__name__, external_stylesheets=[dbc.themes.LUX])

app.layout = dbc.Container([
    dbc.Row([
        dbc.Col([
            dbc.Card([
                dbc.CardImg(src='/assets/linkedin-logo2.png')  # Ensure the path is correct
            ], className='mb-2'),
            dbc.Card([
                dbc.CardBody([
                    dbc.CardLink("Charming Data", target="_blank", href="https://youtube.com/charmingdata")
                ])
            ]),
        ], width=2),
        dbc.Col([
            dbc.Card([
                dbc.CardBody([
                    dcc.DatePickerSingle(
                        id='my-date-picker-start',
                        date=date(2018, 1, 1),
                        className='ml-5'
                    ),
                    dcc.DatePickerSingle(
                        id='my-date-picker-end',
                        date=date(2021, 4, 4),
                        className='mb-2 ml-2'
                    ),
                ])
            ], color="info"),
        ], width=8),
    ], className='mb-2 mt-2'),
    dbc.Row([
        dbc.Col([
            dbc.Card([
                dbc.CardHeader(Lottie(options=options, width="67%", height="67%", url=url_connections)),
                dbc.CardBody([
                    html.H6('Connections'),
                    html.H2(id='content-connections', children="000")
                ], style={'textAlign': 'center'})
            ]),
        ], width=2),
        dbc.Col([
            dbc.Card([
                dbc.CardHeader(Lottie(options=options, width="32%", height="32%", url=url_companies)),
                dbc.CardBody([
                    html.H6('Companies'),
                    html.H2(id='content-companies', children="000")
                ], style={'textAlign': 'center'})
            ]),
        ], width=2),
        dbc.Col([
            dbc.Card([
                dbc.CardHeader(Lottie(options=options, width="25%", height="25%", url=url_msg_in)),
                dbc.CardBody([
                    html.H6('Invites received'),
                    html.H2(id='content-msg-in', children="000")
                ], style={'textAlign': 'center'})
            ]),
        ], width=2),
        dbc.Col([
            dbc.Card([
                dbc.CardHeader(Lottie(options=options, width="53%", height="53%", url=url_msg_out)),
                dbc.CardBody([
                    html.H6('Invites sent'),
                    html.H2(id='content-msg-out', children="000")
                ], style={'textAlign': 'center'})
            ]),
        ], width=2),
        dbc.Col([
            dbc.Card([
                dbc.CardHeader(Lottie(options=options, width="25%", height="25%", url=url_reactions)),
                dbc.CardBody([
                    html.H6('Reactions'),
                    html.H2(id='content-reactions', children="000")
                ], style={'textAlign': 'center'})
            ]),
        ], width=2),
    ], className='mb-2'),
    dbc.Row([
        dbc.Col([
            dbc.Card([
                dbc.CardBody([
                    dcc.Graph(id='line-chart', figure={}, config={'displayModeBar': False}),
                ])
            ]),
        ], width=6),
        dbc.Col([
            dbc.Card([
                dbc.CardBody([
                    dcc.Graph(id='bar-chart', figure={}),
                ])
            ]),
        ], width=4),
    ], className='mb-2'),
    dbc.Row([
        dbc.Col([
            dbc.Card([
                dbc.CardBody([
                    dcc.Graph(id='TBD', figure={}),
                ])
            ]),
        ], width=3),
        dbc.Col([
            dbc.Card([
                dbc.CardBody([
                    dcc.Graph(id='pie-chart', figure={}),
                ])
            ]),
        ], width=3),
        dbc.Col([
            dbc.Card([
                dbc.CardBody([
                    dcc.Graph(id='wordcloud', figure={}),
                ])
            ]),
        ], width=4),
    ], className='mb-2'),
], fluid=True)

# Updating the 5 number cards
@app.callback(
    Output('content-connections', 'children'),
    Output('content-companies', 'children'),
    Output('content-msg-in', 'children'),
    Output('content-msg-out', 'children'),
    Output('content-reactions', 'children'),
    Input('my-date-picker-start', 'date'),
    Input('my-date-picker-end', 'date'),
)
def update_small_cards(start_date, end_date):
    # Connections
    dff_c = df_cnt.copy()
    dff_c = dff_c[(dff_c['Connected On'] >= start_date) & (dff_c['Connected On'] <= end_date)]
    conctns_num = len(dff_c)
    compns_num = len(dff_c['Company'].unique())

    # Invitations
    dff_i = df_invite.copy()
    dff_i = dff_i[(dff_i['Sent At'] >= start_date) & (dff_i['Sent At'] <= end_date)]
    in_num = len(dff_i[dff_i['Direction'] == 'INCOMING'])
    out_num = len(dff_i[dff_i['Direction'] == 'OUTGOING'])

    # Reactions
    dff_r = df_react.copy()
    dff_r = dff_r[(dff_r['Date'] >= start_date) & (dff_r['Date'] <= end_date)]
    reactns_num = len(dff_r)

    return conctns_num, compns_num, in_num, out_num, reactns_num

app.run_server(mode='jupyterlab', port=8005)


KeyError: 'Connected On'

In [22]:
import pandas as pd
print(pd.__version__)


2.2.2


In [12]:
import pandas as pd

# Import App data from csv sheets **************************************
df_cnt = pd.read_csv(r"C:\Users\abhiy\OneDrive\Desktop\project\linked in data\connections.csv", delimiter=",", skipinitialspace=True, on_bad_lines='skip')

# Print the column names to debug
print("Column Names in connections.csv:", df_cnt.columns)

# Strip any leading/trailing spaces from the column names
df_cnt.columns = df_cnt.columns.str.strip()

# Check if the column exists after stripping
if 'Connected On' in df_cnt.columns:
    df_cnt["Connected On"] = pd.to_datetime(df_cnt["Connected On"], format="%d-%b-%y", errors='coerce')
    df_cnt["month"] = df_cnt["Connected On"].dt.month
    df_cnt['month'] = df_cnt['month'].apply(lambda x: calendar.month_abbr[x])
else:
    print("Column 'Connected On' not found in connections.csv")

# Continue with the rest of the code


Column Names in connections.csv: Index(['Notes:'], dtype='object')
Column 'Connected On' not found in connections.csv
